# Getting Started with Fine-Tuning Mistral 7B

This notebook shows you a simple example of how to LoRA finetune Mistral 7B. You can run this notebook in Google Colab with Pro + account with A100 and 40GB RAM.

<a target="_blank" href="https://colab.research.google.com/github/mistralai/mistral-finetune/blob/main/tutorials/mistral_finetune_7b.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


Check out `mistral-finetune` Github repo to learn more: https://github.com/mistralai/mistral-finetune/

## Installation

Clone the `mistral-finetune` repo:


In [1]:
%cd /content/
!git clone https://github.com/mistralai/mistral-finetune.git

/content
fatal: destination path 'mistral-finetune' already exists and is not an empty directory.


Install all required dependencies:

In [2]:
!pip install -r /content/mistral-finetune/requirements.txt

  Using cached torch-2.2.0-cp311-cp311-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached triton-2.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyli

## Model download

In [3]:
!pip install huggingface_hub

In [4]:
# huggingface login
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

! cp -r /root/mistral_models/7B-v0.3 /content/mistral_models
! rm -r /root/mistral_models/7B-v0.3

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

consolidated.safetensors:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

params.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

tokenizer.model.v3:   0%|          | 0.00/587k [00:00<?, ?B/s]

In [6]:
# Alternatively, you can download the model from mistral

# !wget https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar

In [7]:
# !DIR=/content/mistral_models && mkdir -p $DIR && tar -xf mistral-7B-v0.3.tar -C $DIR

In [8]:
!ls /content/mistral_models

7B-v0.3  consolidated.safetensors  params.json	tokenizer.model.v3


## Prepare dataset

To ensure effective training, mistral-finetune has strict requirements for how the training data has to be formatted. Check out the required data formatting [here](https://github.com/mistralai/mistral-finetune/tree/main?tab=readme-ov-file#prepare-dataset).

In this example, let’s use the ultrachat_200k dataset. We load a chunk of the data into Pandas Dataframes, split the data into training and validation, and save the data into the required `jsonl` format for fine-tuning.

In [9]:
%cd /content/

/content


In [10]:
# make a new directory called data
!mkdir -p data

In [11]:
# navigate to this data directory
%cd /content/data

/content/data


In [12]:
#!pip install numpy==2


In [16]:
!pip install torch --force-reinstall

  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.13.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Usin

In [17]:
import torch
import numpy as np


In [18]:
# Install a compatible numpy version (change if a different version is required)
import pandas as pd
import json
# Load your custom JSONL dataset
df = pd.read_json('/content/bargaining_scenarios_enhanced.jsonl', lines=True)

TypeError: Cannot convert numpy.ndarray to numpy.ndarray

In [19]:
import json

# Read the first few lines of the JSONL file
with open('/content/bargaining_scenarios_enhanced.jsonl', 'r') as file:
    for _ in range(5):  # Read first 5 lines
        print(json.loads(file.readline()))


{'messages': [{'role': 'system', 'content': 'Product: Yoga Mat, Price: ₹2000, Min Discount: 5%, Max Discount: 15%, Stock: 7 | User Orders: 3, Refunded: ₹200, Sentiment: Annoyed'}, {'role': 'assistant', 'content': 'Hi there! You’ve got an eye for our premium Yoga Mat. ₹2000 is already a steal!'}, {'role': 'user', 'content': 'I think ₹2000 is too much. How about ₹1500?'}, {'role': 'assistant', 'content': 'Hmm… I like your spirit! How about ₹1800? But only if you buy now.'}, {'role': 'user', 'content': 'Still high. ₹1700?'}, {'role': 'assistant', 'content': 'Alright! I’ll do ₹1700 with a code valid for 10 mins.'}, {'role': 'assistant', 'content': 'Here’s your discount code: YOGAMAT1700'}]}
{'messages': [{'role': 'system', 'content': 'Product: Yoga Mat, Price: ₹2000, Min Discount: 5%, Max Discount: 15%, Stock: 13 | User Orders: 3, Refunded: ₹200, Sentiment: Frustrated'}, {'role': 'assistant', 'content': 'Hi there! You’ve got an eye for our premium Yoga Mat. ₹2000 is already a steal!'}, {'r

In [20]:
import json

# Read the first few lines to inspect the structure of the JSONL file
with open('/content/bargaining_scenarios_enhanced.jsonl', 'r') as file:
    for _ in range(5):  # Read and print the first 5 lines
        print(json.loads(file.readline()))  # Load each line as JSON and print it


{'messages': [{'role': 'system', 'content': 'Product: Yoga Mat, Price: ₹2000, Min Discount: 5%, Max Discount: 15%, Stock: 7 | User Orders: 3, Refunded: ₹200, Sentiment: Annoyed'}, {'role': 'assistant', 'content': 'Hi there! You’ve got an eye for our premium Yoga Mat. ₹2000 is already a steal!'}, {'role': 'user', 'content': 'I think ₹2000 is too much. How about ₹1500?'}, {'role': 'assistant', 'content': 'Hmm… I like your spirit! How about ₹1800? But only if you buy now.'}, {'role': 'user', 'content': 'Still high. ₹1700?'}, {'role': 'assistant', 'content': 'Alright! I’ll do ₹1700 with a code valid for 10 mins.'}, {'role': 'assistant', 'content': 'Here’s your discount code: YOGAMAT1700'}]}
{'messages': [{'role': 'system', 'content': 'Product: Yoga Mat, Price: ₹2000, Min Discount: 5%, Max Discount: 15%, Stock: 13 | User Orders: 3, Refunded: ₹200, Sentiment: Frustrated'}, {'role': 'assistant', 'content': 'Hi there! You’ve got an eye for our premium Yoga Mat. ₹2000 is already a steal!'}, {'r

In [ ]:
import numpy as np

def flatten_json(nested_json):
    """Flatten a nested JSON object into a flat dictionary."""
    flat_json = {}

    def flatten(d, parent_key=''):
        if isinstance(d, dict):
            for k, v in d.items():
                flatten(v, parent_key + k + '_')  # Add underscores to distinguish nested keys
        elif isinstance(d, list):
            for i, v in enumerate(d):
                flatten(v, parent_key + str(i) + '_')
        elif isinstance(d, np.ndarray):
            flat_json[parent_key[:-1]] = d.tolist()  # Convert numpy array to list
        else:
            flat_json[parent_key[:-1]] = d  # Remove trailing underscore
    flatten(nested_json)
    return flat_json

# Now, we will load the JSONL file, flatten it, and convert numpy arrays to lists
data = []
with open('/content/bargaining_scenarios_enhanced.jsonl', 'r') as file:
    for line in file:
        json_data = json.loads(line)  # Load each line as JSON
        flat_data = flatten_json(json_data)  # Flatten the JSON object and handle numpy arrays
        data.append(flat_data)

# Convert the flattened data to a pandas DataFrame
import pandas as pd
df = pd.DataFrame(data)

# Display the first few rows of the DataFrame
df.head()


In [ ]:
import pandas as pd

# Load your custom JSONL dataset
df = pd.read_json('/content/bargaining_scenarios_enhanced.jsonl', lines=True)

In [21]:
df

NameError: name 'df' is not defined

In [ ]:
# split data into training and evaluation
df_train=df.sample(frac=0.95,random_state=200)
df_eval=df.drop(df_train.index)

In [ ]:
# save data into .jsonl files
df_train.to_json("ultrachat_chunk_train.jsonl", orient="records", lines=True)
df_eval.to_json("ultrachat_chunk_eval.jsonl", orient="records", lines=True)

In [ ]:
!ls /content/data

In [22]:
# navigate to the mistral-finetune directory
%cd /content/mistral-finetune/

/content/mistral-finetune


In [23]:
# some of the training data doesn't have the right format,
# so we need to reformat the data into the correct format and skip the cases that don't have the right format:

!python -m utils.reformat_data /content/data/ultrachat_chunk_train.jsonl

In [24]:
# eval data looks all good
!python -m utils.reformat_data /content/data/ultrachat_chunk_eval.jsonl

In [26]:
!pip install weave

ERROR: Operation cancelled by user


In [25]:
!wandb login

wandb: Currently logged in as: 211501016 (211501016-rec) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [28]:
import wandb

# Initialize W&B with a specified project
wandb.init(project="mistral finetune")


In [29]:
# Now you can verify your training yaml to make sure the data is correctly formatted and to get an estimate of your training time.

!python -m utils.validate_data --train_yaml example/7B.yaml


0it [00:00, ?it/s]Validating /content/data/ultrachat_chunk_train.jsonl ...

  0% 0/4750 [00:00<?, ?it/s]
  5% 240/4750 [00:00<00:01, 2398.60it/s]
 10% 480/4750 [00:00<00:01, 2389.04it/s]
 15% 719/4750 [00:00<00:01, 2387.63it/s]
 20% 959/4750 [00:00<00:01, 2390.68it/s]
 25% 1200/4750 [00:00<00:01, 2397.36it/s]
 30% 1443/4750 [00:00<00:01, 2408.11it/s]
 35% 1684/4750 [00:00<00:01, 2402.00it/s]
 41% 1925/4750 [00:00<00:01, 2385.38it/s]
 46% 2164/4750 [00:00<00:01, 2380.27it/s]
 51% 2406/4750 [00:01<00:00, 2390.46it/s]
 56% 2646/4750 [00:01<00:00, 2369.32it/s]
 61% 2887/4750 [00:01<00:00, 2379.59it/s]
 66% 3130/4750 [00:01<00:00, 2394.29it/s]
 71% 3373/4750 [00:01<00:00, 2402.40it/s]
 76% 3615/4750 [00:01<00:00, 2404.72it/s]
 81% 3858/4750 [00:01<00:00, 2410.84it/s]
 86% 4100/4750 [00:01<00:00, 2412.43it/s]
 91% 4342/4750 [00:01<00:00, 2401.81it/s]
100% 4750/4750 [00:01<00:00, 2397.10it/s]
1it [00:01,  1.99s/it]
No errors! Data is correctly formatted!
Stats for /content/data/ultrachat_chun

## Start training

In [30]:
# these info is needed for training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [48]:
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters

config = """
# data
# data
data:
  instruct_data: "/content/data/ultrachat_chunk_train.jsonl"  # Fill
  data: ""  # Optionally fill with pretraining data
  eval_instruct_data: "/content/data/ultrachat_chunk_eval.jsonl"  # Optionally fill

# model
model_id_or_path: "/content/mistral_models"  # Change to downloaded path
lora:
  rank: 64

# optim
seq_len: 200
batch_size: 1
max_steps: 300
optim:
  lr: 6.e-5
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 1
eval_freq: 100
no_eval: False
ckpt_freq: 100

save_adapters: True  # save only trained LoRA adapters. Set to `False` to merge LoRA adapter into the base model and save full fine-tuned model

run_dir: "tune_model"  # Fill

wandb:
  project: "mistral finetune" # your wandb project name
  run_name: "" # your wandb run name
  key: "ea6f25bb96b12a5ef05a9a5141138a5244f7438d" # your wandb api key
  offline: False



"""

# save the same file locally into the example.yaml file
import yaml
with open('example.yaml', 'w') as file:
    yaml.dump(yaml.safe_load(config), file)


In [ ]:
# make sure the run_dir has not been created before
# only run this when you ran torchrun previously and created the /content/test_ultra file
# ! rm -r /content/test_ultra

In [ ]:
#!pip uninstall -y numpy


In [ ]:
#!pip install numpy==1.24.4


In [32]:
import numpy as np
print(np.__version__)


1.23.5


In [34]:
!pip uninstall -y xformers
!pip install xformers


Found existing installation: xformers 0.0.24
Uninstalling xformers-0.0.24:
  Successfully uninstalled xformers-0.0.24
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 46.3 MB/s eta 0:00:00


In [35]:
!pip install torch==2.2.0+cu121
!pip install triton


ERROR: Could not find a version that satisfies the requirement torch==2.2.0+cu121 (from versions: 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0)
ERROR: No matching distribution found for torch==2.2.0+cu121


In [36]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [37]:
import torch
print(torch.version.cuda)


12.4


In [39]:
!rm -rf /content/new_fine_tuned_model


In [46]:
!rm -rf /content/new_fine_tuned_model


In [50]:
# start training

!torchrun --nproc-per-node 1 -m train example.yaml

2025-04-08 11:43:30.493393: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-08 11:43:30.513208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744112610.535871   24182 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744112610.542823   24182 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-08 11:43:30.566184: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## Inference

In [51]:
!pip install mistral_inference

In [ ]:
import torch
from mistral_inference.transformer import Transformer

model = Transformer.from_folder("/content/mistral_models", dtype=torch.bfloat16)

In [54]:
from mistral_inference.transformer import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest


tokenizer = MistralTokenizer.from_file("/content/mistral_models/tokenizer.model.v3")  # change to extracted tokenizer file
model = Transformer.from_folder("/content/mistral_models")  # change to extracted model dir
model.load_lora("/content/mistral-finetune/tune_model/checkpoints/checkpoint_000300/consolidated/lora.safetensors")

completion_request = ChatCompletionRequest(messages=[UserMessage(content="Explain Machine Learning to me in a nutshell.")])

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)

OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 136.88 MiB is free. Process 226004 has 39.41 GiB memory in use. Of the allocated memory 38.80 GiB is allocated by PyTorch, and 124.86 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [58]:
from google.colab import files
files.download("tune_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [59]:
import torch
from mistral_inference.transformer import Transformer

model = Transformer.from_folder("/content/mistral_models", dtype=torch.bfloat16)

OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 136.88 MiB is free. Process 226004 has 39.41 GiB memory in use. Of the allocated memory 38.80 GiB is allocated by PyTorch, and 124.86 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [61]:
import random

discounts = {}  # Dictionary to track user discounts

SYSTEM_PROMPT = """You are a smart bargaining assistant.
You help users negotiate the best price while maintaining a fair and engaging conversation.
Never go below the minimum allowed discount of 10%, and cap maximum discounts at 50%.
Encourage the user to make a deal, but also try to upsell additional products."""

def generate_response(user_id, input_text):
    if user_id not in discounts:
        discounts[user_id] = 10  # Start with a 10% discount

    increase = random.choice([5, 10])  # Randomly increase discount
    discounts[user_id] += increase
    if discounts[user_id] > 50:  # Cap at 50%
        discounts[user_id] = 50

    prompt = f"{SYSTEM_PROMPT}\nUser: {input_text}\nAssistant:"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_length=150, temperature=0.7, top_p=0.9)

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return {"response": response, "discount": discounts[user_id]}
